
# **Apprentissage Fédéré avec Cryptage des Poids**

Ce notebook implémente un exemple simple d'apprentissage fédéré, où un client entraîne un modèle localement, crypte les poids du modèle, et les envoie à un serveur pour agrégation. Le serveur reçoit ces poids cryptés, les décrypte, les agrège avec les poids globaux et met à jour le modèle global.

Le cryptage des poids est effectué pour préserver la confidentialité des données des utilisateurs.


## **Étape 1 : Importation des bibliothèques nécessaires**

In [ ]:
# client.py

import tensorflow as tf
from tensorflow.keras import layers, models
import requests
import numpy as np
import base64
import pickle
import cryptage

def build_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def train_local_model():
    (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
    x_train = x_train.astype('float32') / 255.0
    x_train = np.reshape(x_train, (-1, 28, 28, 1))

    model = build_model()
    model.fit(x_train, y_train, epochs=1, batch_size=32, verbose=2)

    weights = model.get_weights()
    encrypted_weights = cryptage.encrypt_weights(weights)

    url = "http://localhost:5000/update_model"
    response = requests.post(url, json={'weights': encrypted_weights})
    print("Réponse du serveur : ", response.text)

if __name__ == '__main__':
    train_local_model()


In [ ]:
# server.py

from flask import Flask, request, jsonify
import tensorflow as tf
import cryptage

app = Flask(__name__)

def build_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

global_model = build_model()

@app.route('/update_model', methods=['POST'])
def update_model():
    try:
        data = request.get_json()
        if 'weights' not in data:
            return jsonify({"error": "Aucun poids envoyé."}), 400

        encrypted_weights_b64 = data['weights']
        decrypted_weights = cryptage.decrypt_weights(encrypted_weights_b64)

        global_weights = global_model.get_weights()
        if len(decrypted_weights) != len(global_weights):
            return jsonify({"error": "Les tailles des poids sont incompatibles."}), 400

        for i in range(len(global_weights)):
            global_weights[i] = (global_weights[i] + decrypted_weights[i]) / 2

        global_model.set_weights(global_weights)
        global_model.save_weights("model_weights.h5")
        print("Poids du modèle enregistrés.")

        return jsonify({"message": "Modèle mis à jour avec succès."})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# cryptage.py

from cryptography.fernet import Fernet
import base64
import pickle

SHARED_KEY = b'm2eJGGTAoknU0JUPO7O0Ume024lipzhsT1Bf5ag3YE8='
fernet = Fernet(SHARED_KEY)

def encrypt_weights(weights):
    serialized = pickle.dumps(weights)
    encrypted = fernet.encrypt(serialized)
    return base64.b64encode(encrypted).decode('utf-8')

def decrypt_weights(encrypted_b64):
    encrypted = base64.b64decode(encrypted_b64.encode('utf-8'))
    decrypted = fernet.decrypt(encrypted)
    return pickle.loads(decrypted)


In [ ]:
# model.py

import tensorflow as tf

def build_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
model.load_weights("model_weights.h5")
print("Poids rechargés avec succès.")

(_, _), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_test = x_test.astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1)

loss, acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Accuracy sur les données test : {acc:.4f}")
